In [1]:
# 우주타이타닉 문제
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer

gender_df = pd.read_csv('data/names.csv')

train_df = pd.read_csv('data/train.csv')

test_y = pd.read_csv('data/test.csv')

tmp_train_df = train_df.drop(columns=['Transported'])


train_df[["Pass_gggg", 'Pass_pp']] = train_df['PassengerId'].str.split('_', expand=True).astype(int)
test_y[["Pass_gggg", 'Pass_pp']] = test_y['PassengerId'].str.split('_', expand=True).astype(int)
train_df["Family_Size"] = train_df.groupby("Pass_gggg")["PassengerId"].transform("count")
test_y["Family_Size"] = test_y.groupby("Pass_gggg")["PassengerId"].transform("count")

train_means = tmp_train_df.select_dtypes(include=['float64']).mean()  # 수치형 열 평균
train_modes = tmp_train_df.select_dtypes(exclude=['number']).mode().iloc[0]  # 범주형 열 최빈값
for col in train_means.index:
    test_y[col] = test_y[col].fillna(train_means[col])

for col in train_modes.index:
    test_y[col] = test_y[col].fillna(train_modes[col])

# 이렇게하면 130개중 80 개 살림림 
spend_condition = (train_df['CryoSleep'].isnull()) & (train_df['RoomService'] == 0) & (train_df['Spa'] == 0) & (train_df['ShoppingMall'] == 0) & (train_df['FoodCourt'] == 0) \
& (train_df['VRDeck'] == 0)
train_df.loc[spend_condition, 'CryoSleep'] = train_df.loc[spend_condition, 'CryoSleep'].fillna(True)


train_df['Deck'] = train_df['Cabin'].str.split('/').str[0]
train_df['Side'] = train_df['Cabin'].str.split('/').str[2]

#  HomePlanet 결측값 최빈값으로 약 170개 제거
dest_condition = (train_df['HomePlanet'].isnull()) & (train_df['Deck'] == 'A') | (train_df['Deck'] == 'B') | ( train_df['Deck'] == 'C')
train_df.loc[dest_condition, 'HomePlanet'] = train_df.loc[dest_condition, 'HomePlanet'].fillna('Europa')
dest_condition = (train_df['HomePlanet'].isnull()) & ((train_df['Deck'] == 'F'))
train_df.loc[dest_condition, 'HomePlanet'] = train_df.loc[dest_condition, 'HomePlanet'].fillna('Earth')
dest_condition = (train_df['HomePlanet'].isnull()) & ((train_df['Deck'] == 'G'))
train_df.loc[dest_condition, 'HomePlanet'] = train_df.loc[dest_condition, 'HomePlanet'].fillna('Mars')

# 음 ... 
train_df['HomePlanet'] = train_df.groupby('Pass_gggg')['HomePlanet'].transform(
    lambda x: x.fillna(x.mode()[0] if not x.mode().empty else 'Earth')
)

train_df['CryoSleep'] = train_df.groupby('Deck')['CryoSleep'].transform(
    lambda x: x.fillna(x.mode()[0] if not x.mode().empty else False)
)

test_y.loc[spend_condition, 'CryoSleep'] = test_y.loc[spend_condition, 'CryoSleep'].fillna(True)
# test_y.loc[dest_condition, 'HomePlanet'] = test_y.loc[dest_condition, 'HomePlanet'].fillna('Europa')

def fill_missing_values(df):
    # CryoSleep: 소비가 0이면 True
    spend_condition = (df['CryoSleep'].isnull()) & (df[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].sum(axis=1) == 0)
    df.loc[spend_condition, 'CryoSleep'] = True
    
    df['Deck'] = df['Cabin'].str.split('/').str[0]
    for deck, planet in [('A', 'Europa'), ('B', 'Europa'), ('C', 'Europa'), ('F', 'Earth'), ('G', 'Mars')]:
        df.loc[df['HomePlanet'].isnull() & (df['Deck'] == deck), 'HomePlanet'] = planet
    df['HomePlanet'] = df.groupby('Pass_gggg')['HomePlanet'].transform(
        lambda x: x.fillna(x.mode()[0] if not x.mode().empty else 'Earth')
    )
    
    # 숫자형 결측치: KNNImputer
    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
    imputer = KNNImputer(n_neighbors=5)
    df[numeric_cols] = pd.DataFrame(imputer.fit_transform(df[numeric_cols]), columns=numeric_cols, index=df.index)
    
    # 나머지 범주형: 최빈값
    for col in ['Destination', 'VIP']:
        df[col] = df[col].fillna(df[col].mode()[0])
    df['Cabin'] = df['Cabin'].fillna('F/0/S')  # 기본값
    return df

train_df = fill_missing_values(train_df)
test_y = fill_missing_values(test_y)

numeric_cols = train_df.select_dtypes(include=['float64', 'int64']).columns
imputer = KNNImputer(n_neighbors=5)
train_df[numeric_cols] = pd.DataFrame(
    imputer.fit_transform(train_df[numeric_cols]), 
    columns=numeric_cols, 
    index=train_df.index
)

train_df['Age'] = train_df.groupby('Family_Size')['Age'].transform(
    lambda x: x.fillna(x.mean())
)

train_df = train_df.dropna()


train_df['HomePlanet'] = train_df['HomePlanet'].astype('category')
train_df['HomePlanet'] = train_df['HomePlanet'].cat.codes

train_df['Destination'] = train_df['Destination'].astype('category')
train_df['Destination'] = train_df['Destination'].cat.codes

train_df['Side'] = train_df['Side'].astype('category')
train_df['Side'] = train_df['Side'].cat.codes

train_df['Deck'] = train_df['Deck'].astype('category')
train_df['Deck'] = train_df['Deck'].cat.codes

train_df['SideDeck'] = train_df['Deck'] + train_df['Side']
train_df['SideDeck'] = train_df['SideDeck'].astype('category')
train_df['SideDeck'] = train_df['SideDeck'].cat.codes

train_df['VIP'] = train_df['VIP'].astype(int)
train_df['Transported'] = train_df['Transported'].astype(int)
train_df['CryoSleep'] = train_df['CryoSleep'].astype(int)

train_df['Cabin_Num'] = pd.to_numeric(train_df['Cabin'].str.split('/').str[1], errors='coerce')
train_df['Cabin_Num_bin'] = pd.qcut(train_df['Cabin_Num'], q=5, duplicates='drop')


luxury_cols = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

train_df['Total_Spend'] = train_df[luxury_cols].sum(axis=1)
train_df['Spend_Ratio'] = train_df['Total_Spend'] / (train_df['Age'] + 1)  # 0 나누기 방지


test_y['Total_Spend'] = test_y[luxury_cols].sum(axis=1)
test_y['Spend_Ratio'] = test_y['Total_Spend'] / (test_y['Age'] + 1)

test_y['Cabin_Num'] = pd.to_numeric(test_y['Cabin'].str.split('/').str[1], errors='coerce')
test_y['Cabin_Num_bin'] = pd.qcut(test_y['Cabin_Num'], q=5, duplicates='drop')

test_y['Deck'] = test_y['Cabin'].str.split('/').str[0]
test_y['Side'] = test_y['Cabin'].str.split('/').str[2]

test_y['HomePlanet'] = test_y['HomePlanet'].astype('category')
test_y['HomePlanet'] = test_y['HomePlanet'].cat.codes

test_y['Destination'] = test_y['Destination'].astype('category')
test_y['Destination'] = test_y['Destination'].cat.codes

test_y['Side'] = test_y['Side'].astype('category')
test_y['Side'] = test_y['Side'].cat.codes

test_y['Deck'] = test_y['Deck'].astype('category')
test_y['Deck'] = test_y['Deck'].cat.codes

test_y['SideDeck'] = test_y['Deck'] + test_y['Side']
test_y['SideDeck'] = test_y['SideDeck'].astype('category')
test_y['SideDeck'] = test_y['SideDeck'].cat.codes

test_y['VIP'] = test_y['VIP'].astype(int)
test_y['CryoSleep'] = test_y['CryoSleep'].astype(int)

# 로그 변환 적용
train_df['Total_Spend'] = train_df['Total_Spend'].apply(lambda x: np.log1p(x))
train_df['Spend_Ratio'] = train_df['Spend_Ratio'].apply(lambda x: np.log1p(x))

test_y['Total_Spend'] = test_y['Total_Spend'].apply(lambda x: np.log1p(x))
test_y['Spend_Ratio'] = test_y['Spend_Ratio'].apply(lambda x: np.log1p(x))



for col in luxury_cols:
    train_df[col + "_Ratio"] = train_df[col] / (train_df["Total_Spend"] + 1)
    test_y[col + "_Ratio"] = test_y[col] / (test_y["Total_Spend"] + 1)


train_df["Luxury_Spender"] = (train_df["Total_Spend"] > train_df["Total_Spend"].median()).astype(int)
test_y["Luxury_Spender"] = (test_y["Total_Spend"] > test_y["Total_Spend"].median()).astype(int)

# 혼자 온 손님 분류
train_df["Is_Alone"] = (train_df["Family_Size"] == 1).astype(int)
test_y["Is_Alone"] = (test_y["Family_Size"] == 1).astype(int)

# 나이 군집화
train_df["Age_Group"] = pd.cut(train_df["Age"], bins=[0, 12, 18, 30, 50, 100], labels=[0, 1, 2, 3, 4])
test_y["Age_Group"] = pd.cut(test_y["Age"], bins=[0, 12, 18, 30, 50, 100], labels=[0, 1, 2, 3, 4])

# 손님 비율
train_df["Spend_Per_Person"] = train_df["Total_Spend"] / (train_df["Family_Size"] + 1)
test_y["Spend_Per_Person"] = test_y["Total_Spend"] / (test_y["Family_Size"] + 1)

# 극단 값 제거
train_df["Total_Spend"] = np.clip(train_df["Total_Spend"], 0, train_df["Total_Spend"].quantile(0.99))
test_y["Total_Spend"] = np.clip(test_y["Total_Spend"], 0, test_y["Total_Spend"].quantile(0.99))


# 토론 뒤지다가 젠더 찾음
train_df['FirstName'] = train_df['Name'].str.split(' ').str.get(0)
train_df['LastName'] = train_df['Name'].str.split(' ').str.get(1)

test_y['FirstName'] = test_y['Name'].str.split(' ').str.get(0)
test_y['LastName'] = test_y['Name'].str.split(' ').str.get(1)

train_df = train_df.merge(gender_df[['FirstName', 'Gender']], on='FirstName', how='left')
train_df['Gender'] = train_df['Gender'].astype('category').cat.codes

test_y = test_y.merge(gender_df[['FirstName', 'Gender']], on='FirstName', how='left')
test_y['Gender'] = test_y['Gender'].astype('category').cat.codes


/tmp/ipykernel_43539/4288931874.py:26: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_y[col] = test_y[col].fillna(train_modes[col])
/tmp/ipykernel_43539/4288931874.py:31: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_df.loc[spend_condition, 'CryoSleep'] = train_df.loc[spend_condition, 'CryoSleep'].fillna(True)
/tmp/ipykernel_43539/4288931874.py:51: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future beh

In [1]:
from xgboost import XGBClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import GradientBoostingClassifier, VotingClassifier
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier


X = train_df.select_dtypes(include=['number'])
y = train_df.Transported
X = X.drop(columns=['Transported'])

cat_model = CatBoostClassifier(iterations=500, learning_rate=0.04, depth=5, verbose=0)
xgb_model = XGBClassifier(n_estimators=500, learning_rate=0.04, max_depth=3)
svm_model = SVC(probability=True, kernel='rbf', C=1.0)
knn_model = KNeighborsClassifier(n_neighbors=5)


estimators = [
    ('cat', cat_model),
    ('xgb', xgb_model),
    ('svm', svm_model),
    ('knn', knn_model)
]
model = StackingClassifier(estimators=estimators, final_estimator=GradientBoostingClassifier(n_estimators=100))
test_y = test_y[X.columns].fillna(0)
test_y = test_y.select_dtypes(include=['number'])

X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=42)

# 2. 모델 학습
model.fit(X_train, y_train)

# 3. Validation에 대해 확신 높은 예측 선택 (여기서만 생성)
val_proba = model.predict_proba(X_val)[:, 1]
high_conf_mask = (val_proba > 0.95) | (val_proba < 0.05)

pseudo_X = X_val[high_conf_mask]
pseudo_y = (val_proba[high_conf_mask] > 0.55).astype(int)

# 4. 원래 훈련셋과 결합
X_pseudo = pd.concat([X_train, pseudo_X])
y_pseudo = pd.concat([pd.Series(y_train), pd.Series(pseudo_y)])

# 5. 새로 학습
model.fit(X_pseudo, y_pseudo)

# 6. 성능 확인
y_pred = model.predict(X_val)
print("Val accuracy after pseudo labeling:", accuracy_score(y_val, y_pred))

# 7. 최종 테스트셋 예측
final_pred = model.predict(test_y)


# 0.8119575699132112  Spend Ratio 살렸을 때 값. 
# 0.8052073288331726  다 버렸을 때 값 .
# Val accuracy after pseudo labeling: 0.81099324975892
# 0.8066538090646095 
# Val accuracy after pseudo labeling: 0.8085824493731919

NameError: name 'train_df' is not defined

In [14]:
template_df = pd.read_csv('data/test.csv')
dd = template_df['PassengerId']

# int(dd)
ddf = pd.Series(final_pred, name='Transported')
ddf = ddf.map({0: 'False', 1: 'True'})
ds = pd.concat([dd ,ddf], axis=1)
print(ds)
ds.to_csv("result.csv", index=False)

     PassengerId Transported
0        0013_01        True
1        0018_01       False
2        0019_01        True
3        0021_01        True
4        0023_01        True
...          ...         ...
4272     9266_02        True
4273     9269_01        True
4274     9271_01        True
4275     9273_01       False
4276     9277_01        True

[4277 rows x 2 columns]
